In [10]:
import pandas as pd
import re

# Cargar el archivo CSV
df = pd.read_csv("casas_idealista.csv", sep=";", encoding="utf-16")

# Convertir 'caracteristicas_basicas' en una lista de Python, si no está ya en ese formato
df['caracteristicas_basicas'] = df['caracteristicas_basicas'].apply(eval)

# Función para extraer el número de calle
def extraer_numero_calle(titulo):
    # Busca un número en la dirección que esté seguido de coma o espacio
    match = re.search(r',\s?(\d+)', titulo)
    return int(match.group(1)) if match else None

# Aplicar la función para crear la columna 'numero_calle'
df['numero_calle'] = df['titulo'].apply(extraer_numero_calle)

# Función para extraer los datos de 'caracteristicas_basicas'
def extraer_datos(lista):
    datos = {
        'metros_cuadrados': None,
        'habitaciones': None,
        'baños': None,
        'parcela_m2': None,
        'plaza_garaje': None,
        'estado': None,
        'orientacion': None,
        'año_construccion': None,
        'calefaccion': None,
        'planta': None,
        'ascensor': None,
    }

    for item in lista:
        # Solo procesa elementos que contengan características específicas de la vivienda
        if re.search(r'\bm²\b|\bhabitaciones\b|\bbaños\b|garaje|estado|Orientación|Construido en|Calefacción|Planta|ascensor|Parcela', item):
            if re.search(r'\d+\s?m² construidos', item):
                datos['metros_cuadrados'] = int(re.search(r'\d+', item).group())
            elif 'habitaciones' in item:
                datos['habitaciones'] = int(re.search(r'\d+', item).group())
            elif 'baños' in item:
                datos['baños'] = int(re.search(r'\d+', item).group())
            elif 'Parcela de' in item:
                datos['parcela_m2'] = int(re.search(r'\d+', item.replace('.', '')).group())
            elif 'garaje' in item:
                datos['plaza_garaje'] = True
            elif 'Segunda mano' in item or 'para reformar' in item:
                datos['estado'] = item
            elif 'Orientación' in item:
                datos['orientacion'] = item.split(' ')[-1]
            elif 'Construido en' in item:
                datos['año_construccion'] = int(re.search(r'\d+', item).group())
            elif 'Calefacción' in item:
                datos['calefaccion'] = item.split(': ')[-1]
            elif 'Planta' in item:
                datos['planta'] = item
            elif 'ascensor' in item:
                datos['ascensor'] = True

    return datos

# Aplicar la función y expandir los resultados en el DataFrame
datos_df = df['caracteristicas_basicas'].apply(extraer_datos).apply(pd.Series)

# Combinar los datos originales con las nuevas columnas
df = pd.concat([df, datos_df], axis=1)

# Guardar el DataFrame en un nuevo archivo CSV si se desea
df.to_csv('casas_completo.csv', index=False, sep=';')
# Guardar el DataFrame en un nuevo archivo CSV con punto y coma como delimitador



# Mostrar el DataFrame final
print(df.head())




                                              titulo localizacion   precio  \
0          Piso en venta en calle del Duque de Sesto         Goya  1290000   
1          Piso en venta en calle del Duque de Sesto         Goya  1290000   
2                          Ático en venta en Adelfas       Retiro   880000   
3   Chalet pareado en venta en calle Virgen de lo...      Aravaca  2280000   
4   Casa o chalet independiente en venta en calle...   Valdemarín  3500000   

                             caracteristicas_basicas  \
0  [133 m² construidos, 2 habitaciones, 3 baños, ...   
1  [133 m² construidos, 2 habitaciones, 3 baños, ...   
2  [175 m² construidos, 4 habitaciones, 3 baños, ...   
3  [Chalet pareado, 505 m² construidos, 7 habitac...   
4  [Casa o chalet independiente, 749 m² construid...   

                              caracteristicas_extras  numero_calle  \
0   ['Aire acondicionado', 'Consumo:', 'Emisiones:']           NaN   
1   ['Aire acondicionado', 'Consumo:', 'Emisiones:']  